## Notebook to bulk download Planet imagery through the Planet API

Rainey Aberle

Modified from [Planet Developers API Tutorial](https://developers.planet.com/docs/apis/data/) and the Planet Labs Jupyter Notebooks for [API orders tutorials](https://github.com/planetlabs/notebooks/tree/665f165e59f2c3584976ad2dde569c649e631c0b/jupyter-notebooks/orders_api_tutorials).

### 1. Define paths in directory and setup filters for image search. 

#### _MODIFY THIS CELL_

In [ ]:
# -----Define paths in directory
site_name = 'Sperry'
# path to glacier-snow-cover-mapping/
base_path = '/Users/raineyaberle/Research/glacier_snow_cover_mapping/glacier-snow-cover-mapping' 
# path for saving image downloads
out_path = '/Users/raineyaberle/Research/glacier_snow_cover_mapping/study-sites/' + site_name

# -----Area of Interest (AOI)
# Path and filename of your AOI shapefile
aoi_path = '/Users/raineyaberle/Research/glacier_snow_cover_mapping/study-sites/' + site_name + '/AOIs/'
aoi_fn = site_name + '_Glacier_Boundaries_20140907.shp' 

# -----AOI buffer
# Apply buffer to aoi before creating bounding box, image querying, and cropping images
buffer = 500 # meters

# -----Date Range
# Format: 'YYYY-MM-DD'
start_date = '2023-05-01'
end_date = '2023-11-01'

# -----Month Range
# range of months to include, where both start and end months are included
start_month = 5 
end_month = 11

# -----Cloud Cover Filter
# Format: decimal (e.g., 50% max cloud cover = 0.5)
max_cloud_cover = 0.5

# -----Item Type
# See here for possible image ("item") types:
# https://developers.planet.com/docs/apis/data/items-assets/
# PlanetScope Scene: item_type = "PSScene"
# RapidEye Orthorectified Scene: item_type = "REOrthoTile"
item_type = "PSScene"

# -----Asset Type
# Each Item Type has a number of image ("asset") types to choose from.
# See here for the asset types available for your item type:
# https://developers.planet.com/docs/apis/data/items-assets/
# 
# PlanetScope orthorectified 4-band surface reflectance products: asset_type = "ortho_analytic_4b_sr"
# RapidEye orthorectified 5-band surface refelctance products: asset_type = "analytic_sr"
asset_type = "ortho_analytic_4b_sr"

# -----aoi clipping
# Determine whether to clip images to the aoi (True/False)
# This greatly speeds up the ordering and downloading process and decreases the usage of your imagery quota
clip_to_aoi = True

# -----Sentinel-2 Harmonization
# option to harmonize PlanetScope imagery to Sentinel-2
harmonize = True # = True to harmonize

# -----Name of order
# Once requested, this makes it easier to locate the order on your "My Orders" page 
# in order to check the status or to re-download items via the website, etc. 
order_name = site_name + '_' + item_type[0:2] + '_' + start_date.replace('-','') + '-' + end_date.replace('-','')
print('Order name: ' + order_name)

In [ ]:
# -----Import packages
import os
import glob
import json
from getpass import getpass
from planet import Auth, Session, OrdersClient, reporting
import requests
from requests.auth import HTTPBasicAuth
import geopandas as gpd
from shapely import geometry as sgeom
import rioxarray as rxr
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# add path to functions
sys.path.insert(1, os.path.join(base_path, 'functions'))
import pipeline_utils as f
import PlanetScope_orders_utils as orders_utils

### 2. Authenticate your Planet account

Requires your Planet API Key. To find your API Key, Login to your account at [planet.com](https://www.planet.com/) and go to "My Settings". 

If output is `<Repsonse [200]>`, authentication was successful. 

In [ ]:
# set API key as environment variable
API_key = getpass('Enter Planet API Key, then press Enter:')
os.environ['PL_API_KEY'] = API_key

# Setup the API Key stored as the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

# Orders URL
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# Authorize
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

### 3. Reformat AOI for querying

In [ ]:
# -----Read in the shapefile
aoi = gpd.read_file(os.path.join(aoi_path, aoi_fn))
aoi_wgs = aoi.to_crs('EPSG:4326')
# Solver for optimal UTM zone
epsg_utm = f.convert_wgs_to_utm(aoi_wgs.geometry[0].boundary.centroid.coords.xy[0][0], 
                                aoi_wgs.geometry[0].boundary.centroid.coords.xy[1][0])
aoi_utm = aoi.to_crs('EPSG:'+epsg_utm)
aoi_utm_buffer = aoi_utm.buffer(buffer)
# Reproject to WGS84 if necessary
aoi_wgs_buffer = aoi_utm_buffer.to_crs('EPSG:4326')

# -----Convert AOI bounding box to geoJSON format
# Planet only excepts a bounding box as a spatial filter, 
# so we need to convert our aoi to a box (if it is not already). 
aoi_box = {u'type': u'Polygon',
            u'coordinates': [[
               [aoi_wgs_buffer.bounds.minx[0],aoi_wgs_buffer.bounds.miny[0]],
               [aoi_wgs_buffer.bounds.maxx[0],aoi_wgs_buffer.bounds.miny[0]],
               [aoi_wgs_buffer.bounds.maxx[0],aoi_wgs_buffer.bounds.maxy[0]],
               [aoi_wgs_buffer.bounds.minx[0],aoi_wgs_buffer.bounds.maxy[0]],
               [aoi_wgs_buffer.bounds.minx[0],aoi_wgs_buffer.bounds.miny[0]]
            ]]
          }
# Convert AOI geojson to a rasterio Shape for Quick Search function
aoi_box_shape = sgeom.shape(aoi_box)

# -----Plot AOI and bounding box
fig, ax1 = plt.subplots(1, 1, figsize=(8,8))
aoi_wgs.plot(ax=ax1, facecolor='none') # aoi
ax1.plot(*aoi_box_shape.exterior.xy) # aoi box
ax1.set_title('aoi outline and bounding box')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
plt.show()

### 4. Compile filters to create a Quick Search request

Use Quick Search first to grab image IDs before ordering

In [ ]:
# -----Build QuickSearch request and fire off the POST request
im_ids = orders_utils.build_quick_search_request(aoi_box_shape, max_cloud_cover, start_date, end_date, 
                                                 item_type, asset_type, auth=auth)

In [ ]:
# -----Filter image IDs for month range and check if they exist in directory
im_ids_filtered = orders_utils.filter_image_ids(im_ids, start_month, end_month, os.path.join(out_path, 'raw_images'))

### 5. Place order and poll for success

This section compiles the request and places the order with outputs on the order status every ~5 seconds. Download will begin when the console outputs "success". 

In [ ]:
# -----Build new request
request = orders_utils.build_request_with_item_ids(base_path, order_name, aoi_box, clip_to_aoi, harmonize, im_ids_filtered, item_type, asset_type)

# -----Place order and download results
async def create_poll_and_download():
    async with Session(auth=auth) as sess:
        cl = OrdersClient(sess)

        # Use "reporting" to manage polling for order status
        with reporting.StateBar(state='creating') as bar:
            # create order via Orders client
            order = await cl.create_order(request) 
            bar.update(state='created', order_id=order['id'])

            # poll...poll...poll...
            # setting max_attempts=0 means there is no limit on the number of attempts
            await cl.wait(order['id'], callback=bar.update_state, max_attempts=0)

        # if we get here that means the order completed. Yay! Download the files.
        await cl.download_order(order['id'], directory=Path(out_path))
        
# remember: "await" to run the thing
await create_poll_and_download()

### _Optional:_ Plot downloaded images

In [ ]:
# -----Grab output image file names
# set image output folder
im_path = os.path.join(out_path, 'raw_images')
# change directory to im_path
os.chdir(im_path) 
# grab image file names
im_fns = sorted(glob.glob('*SR*.tif'))

# -----Loop through files
for im_fn in im_fns:
    
    # open image
    im = rxr.open_rasterio(im_fn)
    # account for image scalar
    im_scalar = 1e4
    im = im / im_scalar
    # replace no data values with NaN
    im = im.where(im!=-9999)
    
    # plot RGB image
    fig, ax = plt.subplots(1, 1, figsize=(6,6))
    ax.imshow(np.dstack([im.data[2], im.data[1], im.data[0]]), 
              extent=(im.x.data[0]/1e3, im.x.data[-1]/1e3, im.y.data[0]/1e3, im.y.data[-1]/1e3))
    ax.set_xlabel('Easting [km]')
    ax.set_ylabel('Northing [km]')
    ax.set_title(im_fn.split('_')[1][0:10])
    plt.show()